In [121]:
import numpy as np
import pandas as pd
from metadata import data_type
from src.pipe_store import (
    data_loader,
    set_data_types,
    integer_encoder,
)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [122]:
def date_parser_o(x):
    refrence_month = pd.to_datetime('2011-1-1').to_period('M')
    trans_month = pd.to_datetime(x, format='%Y%m').to_period('M')
    return ( trans_month - refrence_month).n #n captures the month number

date_parser = lambda x: pd.to_datetime(x, format='%Y%m')

In [123]:
data_path = '/Users/Danial/Downloads/assesment_file2_churn.csv'
df = (
    data_loader(data_path, parse_dates=['MONTH_PERIOD'], date_parser=date_parser)
    .pipe(set_data_types, data_type)
    # .pipe(integer_encoder, 'CLIENTGROUP', 'ACCOUNTMODEL', 'AGE_CLASS', 'HOMEBANK_COLOUR', 'LOYALITY')
    
)


# df.MONTH_PERIOD = df.MONTH_PERIOD.apply(lambda x: (x - pd.to_datetime('2011-1-1').to_period('M')).n)

Step: data_loader | Shape: (100056, 28) | Computation Time: 0.275846s
Step: set_data_types | Shape: (100056, 28) | Computation Time: 0.12321s


# Missing Values
Variables with missing values are: ACCOUNTMODEL, AGE_CLASS, HOMEBANK_COLOUR, LOYALITY

Missing totally at random -> random sampling from variable distribution

Missing at random -> Predict missing class via Logistic Regression or 

Missing not at random: will not be imputed

In [138]:
df[df.CUSTOMER_ID == 9386078012].AGE_CLASS.values

['Leeftijd_41_54   ', 'Leeftijd_41_54   ', 'Leeftijd_41_54   ', NaN, NaN, ..., 'Leeftijd_41_54   ', 'Leeftijd_41_54   ', 'Leeftijd_41_54   ', 'Leeftijd_41_54   ', 'Leeftijd_41_54   ']
Length: 24
Categories (10, object): ['Leeftijd_12_17   ', 'Leeftijd_18_23   ', 'Leeftijd_24_29   ', 'Leeftijd_30_34   ', ..., 'Leeftijd_55_64   ', 'Leeftijd_65_74   ', 'Leeftijd_75_plus ', 'Leeftijd_onbekend']

In [48]:
len(df.CUSTOMER_ID.unique()) * 24

100056

# Feature Selection

In [49]:
varainces = df.drop(['COMMERCIALLY_CHURNED', 'CHURNED_IND'], axis=1).var().round(2)
print('Top highest variances\n', dict(varainces.sort_values(ascending=False)[0:5]))
print('Top lowest variances\n', dict(varainces.sort_values(ascending=True)[0:5]))
# 'Record_Count', 'TARGET' are discarded from the data set

Top highest variances
 {'CLIENTGROUP': 35.8, 'INVESTED_CAPITAL_CLASS': 13.4, 'SAVINGS_CAPITAL_CLASS': 13.13, 'BUSINESS_VOLUME_CLASS': 6.57, 'CREDIT_CLASS': 5.2}
Top lowest variances
 {'Record_Count': 0.0, 'TARGET': 0.0, 'MORTGAGE_IND': 0.06, 'INSURANCE_LIFE_IND': 0.1, 'PAYMENT_IND': 0.11}


In [55]:
variances = {}
for cust_id in df.CUSTOMER_ID.unique():
    df_id = df[df.CUSTOMER_ID == cust_id]
    variances[cust_id] = dict(df_id.var().round(2))
    

In [108]:
var_client = pd.DataFrame(variances).T

In [114]:
churned_clients = var_client[var_client['CHURNED_IND'] != 0].index.tolist()
com_churned_clients_var = var_client[var_client['COMMERCIALLY_CHURNED'] != 0].index.tolist()
# var_client[(var_client['CHURNED_IND'] == 0) & (var_client['COMMERCIALLY_CHURNED'] != 0)]
# var_client[(var_client['CHURNED_IND'] != 0) & (var_client['COMMERCIALLY_CHURNED'] == 0)]
# var_client[(var_client['CHURNED_IND'] != 0) & (var_client['COMMERCIALLY_CHURNED'] != 0)]

In [139]:
var_client[var_client.AGE_CLASS !=0].AGE_CLASS.shape

(1237,)

In [119]:
df[df.CUSTOMER_ID == 9386078012]

,MONTH_PERIOD,CHURNED_IND,COMMERCIALLY_CHURNED,PAYMENT_IND,SAVING_IND,INVESTMENTS_IND,LENDING_IND,MORTGAGE_IND,INSURANCE_LIFE_IND,INSURANCE_NONLIFE_IND,...,REVENUES_CLASS,PAYMENT_ACTIVITIES_CODE,CLIENTGROUP,ACCOUNTMODEL,AGE_CLASS,HOMEBANK_COLOUR,LOYALITY,TARGET,Record_Count,CUSTOMER_ID
99936,2012-01-01,0,0,1,1,0,1,0,0,0,...,6,3,9,2,4,0,2,99,24,9386078012
99937,2012-02-01,0,0,1,1,0,1,0,0,0,...,6,4,9,2,4,3,1,99,24,9386078012
99938,2012-03-01,0,0,1,1,0,1,0,0,0,...,6,3,9,2,4,3,0,99,24,9386078012
99939,2012-04-01,1,0,0,0,0,0,0,0,0,...,6,0,4,-1,-1,-1,-1,99,24,9386078012
99940,2012-05-01,1,0,0,0,0,0,0,0,0,...,6,1,4,-1,-1,-1,-1,99,24,9386078012
99941,2012-06-01,1,0,0,0,0,0,0,0,0,...,6,0,4,-1,-1,-1,-1,99,24,9386078012
99942,2012-07-01,1,0,0,0,0,0,0,0,0,...,5,0,4,-1,-1,-1,-1,99,24,9386078012
99943,2012-08-01,1,1,0,0,0,0,0,0,0,...,5,0,4,-1,-1,-1,-1,99,24,9386078012
99944,2012-09-01,1,1,0,0,0,0,0,0,0,...,5,0,4,-1,-1,-1,-1,99,24,9386078012
99945,2012-10-01,1,1,0,0,0,0,0,0,0,...,4,0,4,-1,-1,-1,-1,99,24,9386078012


In [ ]:
df.groupby('CUSTOMER_ID').ACCOUNTMODEL.value_counts() # Interesting variable as the model changes after some time
df[df.CUSTOMER_ID == 1276999][['MONTH_PERIOD', 'ACCOUNTMODEL']].head()

In [110]:
var_client

,CHURNED_IND,COMMERCIALLY_CHURNED,PAYMENT_IND,SAVING_IND,INVESTMENTS_IND,LENDING_IND,MORTGAGE_IND,INSURANCE_LIFE_IND,INSURANCE_NONLIFE_IND,CROSS_SELL_SCORE,...,MIN_FEED_CLASS,REVENUES_CLASS,PAYMENT_ACTIVITIES_CODE,CLIENTGROUP,ACCOUNTMODEL,AGE_CLASS,HOMEBANK_COLOUR,LOYALITY,TARGET,Record_Count
6849612538,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.00,...,0.00,0.00,0.04,0.00,0.00,0.00,0.08,0.20,0.0,0.0
8434703737,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.00,...,0.00,0.04,0.00,0.00,0.00,0.00,0.00,0.86,0.0,0.0
5465847147,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.00,...,0.00,0.22,0.17,0.00,0.17,0.00,0.00,0.46,0.0,0.0
6376945954,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.00,...,0.00,4.98,0.25,0.00,0.00,0.00,0.04,0.00,0.0,0.0
2310295193,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.04,0.04,...,0.23,0.04,0.04,0.00,0.00,0.00,0.00,0.58,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9386078012,0.24,0.17,0.24,0.24,0.0,0.24,0.0,0.0,0.00,2.20,...,5.52,1.12,3.17,6.11,2.20,6.11,1.52,1.64,0.0,0.0
6038537607,0.25,0.23,0.25,0.25,0.0,0.00,0.0,0.0,0.00,1.01,...,0.23,0.51,0.00,2.28,2.28,1.01,4.06,2.28,0.0,0.0
2279322201,0.23,0.17,0.23,0.00,0.0,0.00,0.0,0.0,0.00,0.23,...,0.20,0.04,0.22,0.93,2.09,0.93,2.09,0.00,0.0,0.0
107886320,0.04,0.00,0.04,0.04,0.0,0.00,0.0,0.0,0.00,0.17,...,0.00,0.24,0.08,0.67,0.04,0.38,0.00,0.00,0.0,0.0


In [ ]:
df.groupby('CUSTOMER_ID').Record_Count.value_counts() # Interesting variable as the model changes after some time

In [104]:

df_ = df[df.CUSTOMER_ID == churned_clients[8]].sort_values(by='MONTH_PERIOD', ascending=False)
df_[['MONTH_PERIOD', 'CHURNED_IND', 'COMMERCIALLY_CHURNED']]

TypeError: 'set' object is not subscriptable